In [ ]:
### Import Libraries.

import os
import re
import sys
import warnings
import itertools

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.core.display import display, HTML

import scanpy as sc
import anndata
from anndata import AnnData
import scipy.stats as st
import statsmodels.api as sm
import statsmodels.formula.api as smf

import dynamo as dyn
from dynamo.pp import Preprocessor
from dynamo.tools.utils import nearest_neighbors
from dynamo.vf import VectorField
from dynamo.mv import StreamFuncAnim

warnings.filterwarnings('ignore')
dyn.dynamo_logger.main_silence()
%matplotlib inline

In [ ]:
### Configuration.

dyn.configuration.set_figure_params('dynamo', background = 'white')
# or # dyn.configuration.set_figure_params('dynamo', background = 'black')
dyn.configuration.set_pub_style()

In [ ]:
### Load Data.

os.chdir("/folder/")
adata = sc.read_h5ad("adata.h5ad")

adata.X = adata.layers["X_unspliced"].astype(np.float64)

In [ ]:
### Preprocessing

preprocessor = Preprocessor(cell_cycle_score_enable = False)
preprocessor.config_monocle_recipe(adata)
preprocessor.filter_cells_by_outliers_kwargs["keep_filtered"] = True
preprocessor.select_genes_kwargs["n_top_genes"] = 5000
preprocessor.preprocess_adata_monocle(adata)

In [ ]:
### RNA Velocity Inference (Dynamo)

dyn.tl.dynamics(adata, model = 'dynamical', cores = 50)

dyn.tl.gene_wise_confidence(
    adata,
    group = 'Cluster_Column',
    lineage_dict = {'Cluster_Start_1': ['Cluster_End_1', "Cluster_End_2"]}
)
dyn.tl.cell_velocities(
    adata,
    basis = 'pca',
    vkey = 'dynamics',
    method = 'pearson',
    other_kernels_dict = {'transform': 'sqrt'}
)
dyn.tl.confident_cell_velocities(
    adata,
    group = 'Cluster_Column',
    lineage_dict = {'Cluster_Start_1': ['Cluster_End_1', "Cluster_End_2"]}
)

In [ ]:
### Scatter Plot.

dyn.pl.scatters(
    adata,
    color = 'Cluster_Column',
    basis = 'umap',
    show_legend = 'right',
    save_show_or_return = 'save',
    save_kwargs={'ext': 'png', 'prefix': 'umap', 'dpi': 300, 'transparent': False, 'verbose': True}
)

In [ ]:
### Phase Portrait.

dyn.pl.phase_portraits(
    adata,
    basis = 'umap',
    genes = adata.var_names[adata.var.use_for_dynamics][:4],
    figsize=(6, 4),
    color = 'Cluster_Column',
    pointsize = 5
)

In [ ]:
### Phase Portrait. (Specific Feature)

dyn.pl.phase_portraits(
    adata,
    basis = 'umap',
    genes = ['Feature_1'],
    figsize = (6, 4),
    color = 'Cluster_Column',
    pointsize = 5
)

In [ ]:
### Gene Wise Vectors. (Specific Feature)

dyn.pl.cell_wise_vectors(
    adata,
    color = ['Cluster_Column'],
    basis = 'umap',
    show_legend = 'on data',
    quiver_length = 20,
    quiver_size = 2,
    pointsize = 0.9,
    show_arrowed_spines = False
)

In [ ]:
### Streamline Plot.

dyn.pl.streamline_plot(
    adata,
    color = ['Cluster_Column'],
    basis = 'umap',
    show_legend = 'right',
    show_arrowed_spines = True,
    figsize = (5, 4),
    save_show_or_return = 'save',
    save_kwargs = {'ext': 'png', 'prefix': 'Streamline_Plot', 'dpi': 600, 'transparent': True, 'verbose': True}
)

In [ ]:
### Vector Field Reconstruction.

dyn.vf.VectorField(adata, basis = 'umap', pot_curl_div = True)
dyn.pl.plot_energy(adata, basis = 'umap')
dyn.vf.topography(adata, basis = 'umap')
dyn.pl.umap(
    adata,
    color = 'umap_ddhodge_potential',
    frontier = True,
    save_show_or_return = 'show',
    save_kwargs = {'ext': 'png', 'prefix': 'umap_HODGE', 'dpi': 300, 'transparent': False, 'verbose': True}
)

In [ ]:
### Boxplots by Cluster_Column.

cluster_order = [
    "Cluster_1", "Cluster_2",
    "Cluster_3", "Cluster_4",
    "Cluster_5", "Cluster_6"
]

adata.obs['Cluster_Column'] = pd.Categorical(
    adata.obs['Cluster_Column'],
    categories = cluster_order,
    ordered = True
)

df = adata.obs[['Cluster_Column']].copy()
df["DD_Hodge_Potential"] = adata.obsm.get("umap_ddhodge_potential", adata.obs.get("umap_ddhodge_potential"))

plt.figure(figsize = (6, 5))
sns.boxplot(
    data = df,
    x = 'Cluster_Column',
    y = 'DD_Hodge_Potential',
    hue = 'Subclusters_Activation',
    showmeans = True,
    meanprops = {"marker": "o", "markerfacecolor": "red", "markeredgecolor": "black"}
)
plt.xticks(rotation = 90)
plt.title("UMAP DD-Hodge Potential by Cluster_Column")
plt.xlabel("Cluster_Column")
plt.ylabel("DD-Hodge Potential")
plt.tight_layout()
plt.savefig("umap_ddhodge_potential.png", dpi = 800, bbox_inches = 'tight')
plt.show()

In [ ]:
### Additional Vector Field Metrics.

dyn.vf.speed(adata, basis = 'pca')
dyn.vf.curl(adata, basis = 'umap')
dyn.vf.divergence(adata, basis = 'pca')
dyn.vf.acceleration(adata, basis = 'pca')
dyn.vf.curvature(adata, basis = 'pca')

In [ ]:
### Integrative Visualization.

fig, axes = plt.subplots(ncols = 2, nrows = 2, constrained_layout = True, figsize = (12, 8))
dyn.pl.cell_wise_vectors(adata, color = 'speed_pca', pointsize = 0.5, alpha = 0.7, ax = axes[0, 0], quiver_length = 6, quiver_size = 6, save_show_or_return = 'return')
dyn.pl.grid_vectors(adata, color = 'divergence_pca', ax = axes[0, 1], quiver_length = 12, quiver_size = 12, save_show_or_return = 'return')
dyn.pl.streamline_plot(adata, color = 'acceleration_pca', ax = axes[1, 0], save_show_or_return = 'return')
dyn.pl.streamline_plot(adata, color = 'curvature_pca', ax = axes[1, 1], save_show_or_return = 'return')
plt.show()

In [ ]:
### Statistical Analysis.

df_stats = adata.obs[['umap_ddhodge_potential', 'Cluster_Column', 'Sample_ID']].dropna()
model = smf.mixedlm(
    "umap_ddhodge_potential ~ C(Cluster_Column, Treatment(reference = 'Cluster_1'))",
    data = df_stats,
    groups = df_stats["Sample_ID"]
)
result = model.fit()
print(result.summary())